In [1]:
#Import libraries.
import time
import multiprocessing 
from multiprocessing import Pool
from multiprocessing import Semaphore
import asyncio
import nest_asyncio
nest_asyncio.apply()
import cdsapi
import os
import shutil
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import matplotlib.patches as mpatches
import cmocean

#To fix the probelm with the basemap package in python.
#Before, the line "from matplotlib.cbook import dedent" in "mpl.toolkits/proj.py" should be replaced by "from inspect import cleandoc as dedent"
os.environ['PROJ_LIB'] = '/home/juan-david/anaconda3/envs/ds/share/proj'
from mpl_toolkits.basemap import Basemap
%run -i '/home/juan-david/Documents/data_science/UGA_phd/documents/ERA5_data_download/retrieving_data_fonctions_aide.py'

In [2]:
#Initiate era5 API.
#The following is a good tutorial video. (https://www.youtube.com/watch?v=AmF1nn7o6Hc)
#1. Setting the configuration file into the home folder.
#Get current path.
current_path = os.getcwd()

#Get home path.
home_path = os.path.expanduser('~')

#Copy file in current directory to home directory.
shutil.copyfile(current_path+'/.cdsapirc', home_path+'/.cdsapirc')

'/home/juan-david/.cdsapirc'

In [3]:
#2.Accessing to data.
#Create client acces in API.
c = cdsapi.Client()

#Obtain objects for era5 data retrieval.
d = get_dictionaries_era5_data_retrieval()
hours = get_hours_era5_data_retrieval()
days = get_days_era5_data_retrieval()

#Before accessing to data, the clients should have agreed to the required terms and conditions in :
#https://cds.climate.copernicus.eu/cdsapp/#!/terms/licence-to-use-copernicus-products
    
#https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview.

#The ERA5 dataset contains one (hourly, 31 km) high resolution realisation (referred to as "reanalysis" or "HRES") and a reduced resolution ten member ensemble (referred to as "ensemble" or "EDA")

#It is important to define in the query the following aspects (as in https://cds.climate.copernicus.eu/cdsapp#!/dataset/) :
    #variable
    #pressure_level
    #product_type
    #year
    #month
    #day
    #time
    #format

In [11]:
#Documentation for examples.
#To check datasets parameters. (Allowable keyword parameters for each dataset are)
#https://confluence.ecmwf.int/display/CKB/Climate+Data+Store+%28CDS%29+API+Keywords
#To check available variables and datasets.
#https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation

#3.Example of API use.
#Downloading pressure level temperature, and storing that as a nc file.
#The results of the query will appear in the current working directory with the name specified in the second method argument ("my_downloaded_data.nc").
#desired_file_name = 'era5_data' 
desired_file_name = 'era5_data'
c.retrieve("reanalysis-era5-pressure-levels",
{
"variable": "temperature",
"pressure_level": "1000",
"product_type": "reanalysis",
"year": "2022",
"month": "01",
"day": "01",
"time": "12:00",
"format": "netcdf"
}, desired_file_name)

/home/juan-david/anaconda3/envs/ds/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
2022-05-31 23:46:37,282 INFO Welcome to the CDS
2022-05-31 23:46:37,282 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
/home/juan-david/anaconda3/envs/ds/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
2022-05-31 23:46:37,376 INFO Downloading https://download-0004-clone.copernicus-climate.eu/cache-compute-0004/cache

Result(content_length=2086232,content_type=application/x-netcdf,location=https://download-0004-clone.copernicus-climate.eu/cache-compute-0004/cache/data0/adaptor.mars.internal-1654033583.2573025-15850-7-35bb1c50-7dbd-472b-81fb-9e075ef4ecbc.nc)

In [14]:
path = '/home/juan-david/Documents/data_science/UGA_phd/documents/ERA5_data_download/ERA5_data_2022_01'

In [8]:
#Retrieven data in an hour.
retrieve_ERA5_data_in_hour(c,d,'reanalysis-era5-pressure-levels','2019','01','01','12:00',path)

/home/juan-david/anaconda3/envs/ds/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
2022-05-30 23:57:20,893 INFO Welcome to the CDS
2022-05-30 23:57:20,894 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
/home/juan-david/anaconda3/envs/ds/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
2022-05-30 23:57:21,038 INFO Request is completed
2022-05-30 23:57:21,040 INFO Downloading https://download-0011-cl

In [17]:
#Retrieve data in a day.
retrieve_ERA5_data_in_day(c,d,'reanalysis-era5-pressure-levels','2022','01','12', hours, path)

/home/juan-david/anaconda3/envs/ds/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
2022-06-01 00:32:15,320 INFO Welcome to the CDS
2022-06-01 00:32:15,321 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels


12


/home/juan-david/anaconda3/envs/ds/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/juan-david/anaconda3/envs/ds/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/juan-david/anaconda3/envs/ds/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/

In [13]:
#Retrieve data in a month.
retrieve_ERA5_data_in_month(c,d,'reanalysis-era5-pressure-levels','2022','01', days, hours)

01040302





/home/juan-david/anaconda3/envs/ds/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/juan-david/anaconda3/envs/ds/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/juan-david/anaconda3/envs/ds/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/

05

/home/juan-david/anaconda3/envs/ds/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


0708

...


31

/home/juan-david/anaconda3/envs/ds/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


2022-06-01 00:20:24,571 INFO Welcome to the CDS
2022-06-01 00:20:24,591 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
/home/juan-david/anaconda3/envs/ds/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/home/juan-david/anaconda3/envs/ds/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
2022-06-01 00:20:24,667 INFO Welcome to the CDS
/home/juan-david/anaconda3/envs/ds/lib/python3.7/site-packages/urll

Request is not valid


KeyboardInterrupt: 

In [ ]:
import time

In [ ]:
import multiprocessing 
from multiprocessing import Pool
from multiprocessing import Semaphore

In [ ]:
#In parallel.
start = time.time()
retrieve_in_month_imp(10)
end = time.time()
print(end - start)

In [ ]:
#4.Plotting obtained information with python.
my_example_nc_file =  'ERA5_data_2019_01_01/r_era5_temp_2019_01_01_10h.nc'
fh = Dataset(my_example_nc_file)

#Reading some variables of the file.
#Important: The latitude and longitude coordinates of the ERA5 data are already referenced with respect to the WGS84.
temp = fh.variables['t'][:]
time = fh.variables['time'][:]
lons = fh.variables['longitude'][:]
lats = fh.variables['latitude'][:]

In [ ]:
type(fh)

In [ ]:
print(temp.shape) #int16 t(time, latitude, longitude)
print(time.shape)
print(lons.shape)
print(lats.shape)

In [ ]:
#Closing the file after extracting the iimportant information.
fh.close()

In [ ]:
#Accesing data.
#TIme.
print(np.ma.getdata(time).shape)
print(np.ma.getdata(time))

#Temperature.
print(np.ma.getdata(temp).shape)
print(np.ma.getdata(temp))

#Lats
print(np.ma.getdata(lats).shape)
print(np.ma.getdata(lats))
#Longs.
print(np.ma.getdata(lons).shape)
print(np.ma.getdata(lons))

In [ ]:
#Defining base map and coordinates.
m = Basemap(projection='merc',llcrnrlat=-80,urcrnrlat=80,llcrnrlon=-180,urcrnrlon=180)
lons,lats= np.meshgrid(lons-180,lats) # for this dataset, longitude is 0 through 360, so you need to subtract 180 to properly display on map
x,y = m(lons,lats)

In [ ]:
#Plotting.
fig = plt.figure(figsize = (15,10))
m.drawcoastlines()
m.drawcountries() 
m.drawmapboundary()

temp = m.contourf(x,y,np.squeeze(temp), cmap=plt.cm.jet)
cb = m.colorbar(temp,"bottom", size="5%", pad="2%")
plt.title('2m Temperature')
cb.set_label('Temperature (K)')
plt.title("Temperature", fontsize=20)